# Utilities for Extracting and Creating Resources


## Import Python Modules and initialize Global Variables

In [12]:
import datetime
import os
import rdflib
import urllib.parse
import xml.etree.ElementTree as etree

rdf_template = """<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF xmlns:dcterms="http://purl.org/dc/terms/"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:skos="http://www.w3.org/2004/02/skos/core#"
         xmlns:rdact="http://rdaregistry.info/termList/RDACarrierType/">
</rdf:RDF>"""

ontology_html_template ="""<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
    <h1>{label}</h1>
    <p>
      Documentation at <a href="/ontologies/vocabularies/{html_doc}">{label}</a>.
    </p>
    <ul>
    {entities_list}
    </ul>
</body>
</html>"""

html_template = """<!doctype html>
<html lang="en">
<head>
  <link href="/css/owl.css" rel="stylesheet" type="text/css" />
  <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
  <link href="/css/rec.css" rel="stylesheet" type="text/css" />
  <link href="/css/extra.css" rel="stylesheet" type="text/css" />
  <link rel="shortcut icon" href="/css/favicon.ico" />
</head>
<body>
  <h1>{label}</h1>
  <p>
      Documentation at <a href='/ontologies/vocabularies/{html_doc}#{name}'>{label}</a>.
      Other versions:
      <ul>
          <li><a href="/ontologies/vocabularies/audio_carrier/{name}.rdf">RDF XML</a></li>
          <li><a href="/ontologies/vocabularies/audio_carrier/{name}.ttl">Turtle</a></li>
      </ul>
  </p>
</body>
</html>"""

## Supporting functions

In [13]:
def init_ontology(**kwargs):
    rdf_path = kwargs.get("rdf_path")
    html_doc_file = kwargs.get("html_doc_file")
    start = datetime.datetime.utcnow()
    print("{} Staring ontology initialization".format(start.isoformat()))
    if not os.path.exists(rdf_path):
        raise ValueError("{} rdf_path doesn't exist".format(rdf_path))
    ontology_xml = etree.parse(rdf_path)
    entities = ontology_xml.findall("{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description")
    ontology_entity = entities.pop(0)
    ontology_label = ontology_entity.find("{http://www.w3.org/2000/01/rdf-schema#}label").text
    ontology_uri = urllib.parse.urlparse(ontology_entity.get("{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about"))
    ontology_path = os.path.abspath("..{0}".format(ontology_uri.path))
    if not os.path.exists(ontology_path):
        os.makedirs(ontology_path)

    entity_list_html = ''
    for i,row in enumerate(entities):
        if i > 0 and not i%10:
            print("{:2}".format(i), end="")
        else:
            print(".", end="")
        entity_list_html += "{}\n".format(
            init_entity(entity=row,
                        ontology_name=ontology_label,
                        ontology_path=ontology_path,
                        html_doc_file=html_doc_file))
    raw_html = ontology_html_template.format(**{
            "label": ontology_label,
            "html_doc": html_doc_file,
            "entities_list": entity_list_html})
    with open(os.path.join(os.path.abspath(ontology_path), "index.html"), "w+") as fo:
        fo.write(raw_html)
    end = datetime.datetime.utcnow()
    print("{} - Finished {}, total time {:,} minutes".format(
        end.isoformat(),
        ontology_label,
        (end-start).seconds / 60.0 ))
    
def init_entity(**kwargs):
    entity = kwargs.get('entity')
    ontology_name = kwargs.get("ontology_name")
    ontology_path = kwargs.get("ontology_path")
    html_doc_file = kwargs.get("html_doc_file")
    uri = urllib.parse.urlparse(
        entity.attrib.get("{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about"))
    label = entity.find("{http://www.w3.org/2004/02/skos/core#}prefLabel").text
    name = uri.path.split("/")[-1]

    # Make sub-directory if it doesn't exist
    entity_path = os.path.join(os.path.abspath(ontology_path), name)
    if not os.path.exists(entity_path):
        os.makedirs(entity_path)
    # Create an HTML file that redirects to PMOAudioCarrier.html
    html_path = os.path.join(os.path.abspath(entity_path), "index.html")
    if not os.path.exists(html_path):
        with open(html_path, "w+") as fo:
            fo.write(html_template.format(**{ "html_doc": html_doc_file,
                                              "label": label, 
                                              "name": name }))
    graph = rdflib.ConjunctiveGraph()
    graph.parse(data=rdf_template, format='xml')
    graph.parse(data=etree.tostring(entity), format='xml')
    # Writes an RDF XML file if it doesn't exist
    rdf_xml_path = os.path.join(os.path.abspath(ontology_path), "{}.rdf".format(name))
    save_rdf(graph, rdf_xml_path, "xml")
    # Writes a RDF Turtle file
    rdf_ttl_path = os.path.join(os.path.abspath(ontology_path), "{}.ttl".format(name))
    save_rdf(graph, rdf_ttl_path, "ttl") 
    print("\t{time} Finished initializing {label}".format(
          **{ "time": datetime.datetime.utcnow().isoformat(),
            "label": label }))
    
    return """<li><a href="{}">{}</li>""".format(uri.path,label)
    
def save_rdf(graph, path, serialize_as):
    if not os.path.exists(path):
        print("\t\t{} Saving {}, {} triples as {}".format(datetime.datetime.utcnow().isoformat(),
                                                  path,
                                                  len(graph),
                                                  serialize_as))
        with open(path, "wb+") as fo:
            fo.write(graph.serialize(format=serialize_as))
    print(".", end="")

### Audio Carrier Results

In [14]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMOAudioCarrier.rdf",
    html_doc_file="PMOAudioCarrier.html")

2019-04-03T14:29:31.102231 Staring ontology initialization
.		2019-04-03T14:29:31.106667 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/audio_carrier/cd_audio.rdf, 8 triples as xml
.		2019-04-03T14:29:31.108020 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/audio_carrier/cd_audio.ttl, 8 triples as ttl
.	2019-04-03T14:29:31.110002 Finished initializing CD Audio
.		2019-04-03T14:29:31.112901 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/audio_carrier/dat.rdf, 8 triples as xml
.		2019-04-03T14:29:31.115346 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/audio_carrier/dat.ttl, 8 triples as ttl
.	2019-04-03T14:29:31.117434 Finished initializing DAT
.		2019-04-03T14:29:31.120357 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/audio_carrier/disc_45.rdf, 8 triples as xml
.		2019-04-03T14:29:31.121611 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontolo

### Disc Cutting

In [15]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMODiscCutting.rdf",
    html_doc_file="PMODiscCutting.html")

2019-04-03T14:31:37.498616 Staring ontology initialization
.		2019-04-03T14:31:37.502889 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/disc_cutting/lateral_cut.rdf, 7 triples as xml
.		2019-04-03T14:31:37.504210 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/disc_cutting/lateral_cut.ttl, 7 triples as ttl
.	2019-04-03T14:31:37.505683 Finished initializing lateral cut
.		2019-04-03T14:31:37.508747 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/disc_cutting/vertical_cut.rdf, 8 triples as xml
.		2019-04-03T14:31:37.510283 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/disc_cutting/vertical_cut.ttl, 8 triples as ttl
.	2019-04-03T14:31:37.512066 Finished initializing vertical cut
2019-04-03T14:31:37.512785 - Finished PMO Disc Cutting Method Vocabulary, total time 0.0 minutes


### Encoding Format

In [16]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMOEncodingFormat.rdf",
              html_doc_file="PMOEncodingFormat.html")

2019-04-03T14:32:43.957418 Staring ontology initialization
.		2019-04-03T14:32:43.962129 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/encoding_format/Aac.rdf, 8 triples as xml
.		2019-04-03T14:32:43.963588 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/encoding_format/Aac.ttl, 8 triples as ttl
.	2019-04-03T14:32:43.965174 Finished initializing AAC
.		2019-04-03T14:32:43.968129 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/encoding_format/Aiff.rdf, 7 triples as xml
.		2019-04-03T14:32:43.969599 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/encoding_format/Aiff.ttl, 7 triples as ttl
.	2019-04-03T14:32:43.971625 Finished initializing AIFF
.		2019-04-03T14:32:43.975282 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/encoding_format/Flac.rdf, 8 triples as xml
.		2019-04-03T14:32:43.977071 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/

### Medium Part Type

In [17]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMOMediumPartType.rdf",
              html_doc_file="PMOMediumPartType.html")

2019-04-03T14:36:20.886397 Staring ontology initialization
.		2019-04-03T14:36:20.890811 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/medium_part_type/ad_lib.rdf, 8 triples as xml
.		2019-04-03T14:36:20.892056 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/medium_part_type/ad_lib.ttl, 8 triples as ttl
.	2019-04-03T14:36:20.893767 Finished initializing ad lib
.		2019-04-03T14:36:20.896790 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/medium_part_type/amplified.rdf, 7 triples as xml
.		2019-04-03T14:36:20.898422 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/medium_part_type/amplified.ttl, 7 triples as ttl
.	2019-04-03T14:36:20.900088 Finished initializing amplified
.		2019-04-03T14:36:20.902990 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/medium_part_type/obbligato.rdf, 7 triples as xml
.		2019-04-03T14:36:20.904546 Saving /Users/jpnelson/2019/

### Playing Speed

In [18]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMOPlayingSpeed.rdf",
              html_doc_file="PMOPlayingSpeed.html")

2019-04-03T14:38:05.311442 Staring ontology initialization
.		2019-04-03T14:38:05.315796 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/playing_speed/fpm_12.rdf, 8 triples as xml
.		2019-04-03T14:38:05.317126 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/playing_speed/fpm_12.ttl, 8 triples as ttl
.	2019-04-03T14:38:05.318791 Finished initializing 12 fpm
.		2019-04-03T14:38:05.321942 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/playing_speed/ips.9.rdf, 9 triples as xml
.		2019-04-03T14:38:05.323475 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/playing_speed/ips.9.ttl, 9 triples as ttl
.	2019-04-03T14:38:05.325411 Finished initializing 15/16 ips
.		2019-04-03T14:38:05.328827 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/playing_speed/Ips1.8.rdf, 8 triples as xml
.		2019-04-03T14:38:05.330048 Saving /Users/jpnelson/2019/PerformedMusicOntology/ont

### Tonel Center

In [19]:
init_ontology(rdf_path="/Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/PMOTonalCenter.rdf",
               html_doc_file="PMOTonalCenter.html")

2019-04-03T14:55:24.434985 Staring ontology initialization
.		2019-04-03T14:55:24.439499 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/tonal_center/C.rdf, 6 triples as xml
.		2019-04-03T14:55:24.440527 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/tonal_center/C.ttl, 6 triples as ttl
.	2019-04-03T14:55:24.441817 Finished initializing C
.		2019-04-03T14:55:24.445277 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/tonal_center/C_flat.rdf, 7 triples as xml
.		2019-04-03T14:55:24.446885 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/tonal_center/C_flat.ttl, 7 triples as ttl
.	2019-04-03T14:55:24.448559 Finished initializing C♭
.		2019-04-03T14:55:24.451280 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabularies/tonal_center/C_double_flat.rdf, 7 triples as xml
.		2019-04-03T14:55:24.452473 Saving /Users/jpnelson/2019/PerformedMusicOntology/ontologies/vocabulari